In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from fig_style import *

import sys
sys.path.insert(0,'../')
from disk_model import DiskFitting, DiskImage

import astropy.constants as const
au = const.au.cgs.value

In [2]:
with open("../data/fitted_systems/fit_1mm_Q1d5_age1e5.pkl","rb") as f:
    Ds = pickle.load(f)

In [3]:
import astropy
data_all = astropy.table.Table.read("../data/VANDAM_T20_properties.txt", format="ascii")
data_all.add_index('Source')

In [4]:
import scipy
def fit_gaussian(D, i_obs=0):
    I = D.disk_image_list[i_obs]
    I.generate_mock_observation(R=D.disk_model.R, I=D.disk_model.I_obs[i_obs], cosI=D.cosI)
    img_model = I.img_model.copy()
    def generate_gaussian_image(x):
        A, sigma, cosI = x
        R = np.linspace(0,5*sigma,100)
        Rc = (R[1:]+R[:-1])/2
        IR = A*np.exp(-(Rc/sigma)**2/2)
        I.generate_mock_observation(R=R, I=IR, cosI=cosI)
        return I.img_model
    A0 = np.amax(img_model)/(1e23*I.beam_area)
    i = data_all.loc_indices[D.source_name]
    if i_obs == 0:
        sigma0 = data_all[i]['RdiskA']/2
    else:
        sigma0 = data_all[i]['RdiskV']/2
    if np.ma.is_masked(sigma0):
        sigma0 = np.sqrt(np.sum(img_model)/(1e23*I.beam_area)/A0) * I.au_per_pix
    x0 = np.array([A0, sigma0*au, D.cosI])
    ig0 = generate_gaussian_image(x0)
    def compare_img(x_rel):
        img_gaussian = generate_gaussian_image(x_rel*x0)
        return np.sum((img_gaussian-img_model)**2)/np.sum(img_model**2)
    #return generate_gaussian_image(x0)
    res = scipy.optimize.minimize(compare_img, x0=[1,1,1])
    return res, generate_gaussian_image(res.x*x0), img_model, (res.x*x0)

In [ ]:
sigma_a = {}
sigma_v = {}
for D in Ds:
    res, i_g, i_m, x = fit_gaussian(D)
    if x[2]>1:
        x[1] = x[1]*x[2]
        x[2] = 1/x[2]
    sigma_a[D.source_name] = x[1]/au
for D in Ds:
    res, i_g, i_m, x = fit_gaussian(D, 1)
    if x[2]>1:
        x[1] = x[1]*x[2]
        x[2] = 1/x[2]
    sigma_v[D.source_name] = x[1]/au

In [ ]:
sigma_a_T20 = {}
sigma_v_T20 = {}
sigma_a_T20_lower = {}
sigma_v_T20_lower = {}
for D in Ds:
    i = data_all.loc_indices[D.source_name]
    sigma_a_T20[D.source_name] = data_all[i]['RdiskA']/2
    sigma_v_T20[D.source_name] = data_all[i]['RdiskV']/2
    Rlower = 1 if np.ma.is_masked(data_all[i]['RdiskA']) else np.nan
    sigma_a_T20_lower[D.source_name] = Rlower
    Rlower = 1 if np.ma.is_masked(data_all[i]['RdiskV']) else np.nan
    sigma_v_T20_lower[D.source_name] = Rlower

# Compare apparant disk size

In [ ]:
plt.figure(figsize=(4.5,4.5))
plt.plot([1,500],[1,500],'k:',label='_nolegend_', zorder=-100)

x,y,yl = [],[],[]
for D in Ds:
    x.append(sigma_a[D.source_name]*2)
    y.append(sigma_a_T20[D.source_name]*2)
    yl.append(sigma_a_T20_lower[D.source_name]*6)

alpha, s = 1, 15
plt.scatter(y,x, facecolors='tab:blue', edgecolors='None', alpha=alpha, s=s)
plt.scatter(yl,x, facecolors='tab:blue', edgecolors='None', alpha=alpha, s=s, marker='<',label='_nolegend_')
    
x,y,yl = [],[],[]
for D in Ds:
    x.append(sigma_v[D.source_name]*2)
    y.append(sigma_v_T20[D.source_name]*2)
    yl.append(sigma_v_T20_lower[D.source_name]*6)
alpha, s = 1, 12
plt.scatter(y,x, facecolors='None', edgecolors='tab:orange', alpha=alpha, s=s)
plt.scatter(yl,x, facecolors='None', edgecolors='tab:orange', alpha=alpha, s=s, marker='<',label='_nolegend_')

plt.xscale('log')
plt.yscale('log')
plt.gca().set_aspect('equal','box')
plt.xlim(5,300)
plt.ylim(5,300)

plt.legend(['0.87 mm (ALMA)','9 mm (VLA)'],frameon=False,loc=2)

set_ticks()

plt.xlabel(r'$R_{\rm 2\sigma}$ from observation [au]')
plt.ylabel(r'$R_{\rm 2\sigma}$ from our model [au]')

#plt.savefig('../figures/R2sigma_comp.pdf',bbox_inches='tight')

# For the subsample with chi^2<1 (not in paper)

In [ ]:
from get_chi_sq import get_mean_chisq_mult
chi_sq = get_mean_chisq_mult(Ds)

In [ ]:
low_chi_sq = (chi_sq<=1)

In [ ]:
plt.figure(figsize=(4.5,4.5))
plt.plot([1,500],[1,500],'k:',label='_nolegend_', zorder=-100)

x,y,yl = [],[],[]
for D in np.array(Ds)[low_chi_sq]:
    x.append(sigma_a[D.source_name]*2)
    y.append(sigma_a_T20[D.source_name]*2)
    yl.append(sigma_a_T20_lower[D.source_name]*6)

alpha, s = 1, 15
plt.scatter(y,x, facecolors='tab:blue', edgecolors='None', alpha=alpha, s=s)
plt.scatter(yl,x, facecolors='tab:blue', edgecolors='None', alpha=alpha, s=s, marker='<',label='_nolegend_')
    
x,y,yl = [],[],[]
for D in np.array(Ds)[low_chi_sq]:
    x.append(sigma_v[D.source_name]*2)
    y.append(sigma_v_T20[D.source_name]*2)
    yl.append(sigma_v_T20_lower[D.source_name]*6)
alpha, s = 1, 12
plt.scatter(y,x, facecolors='None', edgecolors='tab:orange', alpha=alpha, s=s)
plt.scatter(yl,x, facecolors='None', edgecolors='tab:orange', alpha=alpha, s=s, marker='<',label='_nolegend_')

plt.xscale('log')
plt.yscale('log')
plt.gca().set_aspect('equal','box')
plt.xlim(5,300)
plt.ylim(5,300)

plt.legend(['0.87 mm (ALMA)','9 mm (VLA)'],frameon=False,loc=2)

set_ticks()

plt.xlabel(r'$R_{\rm 2\sigma}$ from observation [au]')
plt.ylabel(r'$R_{\rm 2\sigma}$ from our model [au]')

#plt.savefig('../figures/R2sigma_comp.pdf',bbox_inches='tight')